In [25]:
from safetensors.torch import load_file
import torch
import os
import glob
from datasets import Dataset, load_from_disk, load_dataset

# Modify this to your shard path pattern
shard_pattern = "/mnt/pdata/sonia111/ArmoRM/embedding_all_prompt/FsfairX-LLaMA3-RM-v0.1/ArmoRM-Multi-Objective-Data-v0.1-*.safetensors"
shard_files = sorted(glob.glob(shard_pattern))

embeddings, prompt_embeddings, labels = [], [], []

for path in shard_files:
    data = load_file(path)
    embeddings.append(data['embeddings'])
    prompt_embeddings.append(data['prompt_embeddings'])
    labels.append(data['labels'])

embeddings = torch.cat(embeddings, dim=0)
prompt_embeddings = torch.cat(prompt_embeddings, dim=0)
labels = torch.cat(labels, dim=0)
root = "/mnt/pdata/knk25/active_pref_learning"

In [26]:
from datasets import Dataset, load_from_disk, load_dataset
dets = "/mnt/pdata/sonia111/ArmoRM/embedding_all_prompt/armo_pairs_pref_concepts"
data_labels = load_from_disk(dets)

In [27]:
labels_df = data_labels.to_pandas()[['idx_a', 'idx_b', 'relative_concept_labels']].copy()

In [28]:
reward_cols = [
    "helpsteer-helpfulness", "helpsteer-correctness", "helpsteer-coherence", "helpsteer-complexity", "helpsteer-verbosity",
    "ultrafeedback-overall_score", "ultrafeedback-instruction_following", "ultrafeedback-truthfulness", "ultrafeedback-honesty", "ultrafeedback-helpfulness",
    "beavertails-is_safe", "prometheus-score",
    "argilla-overall_quality", "argilla-judge_lm",
    "code-complexity", "code-style", "code-explanation", "code-instruction-following", "code-readability"
]

# Add a new column with the same list for every row
labels_df['concept_names'] = [reward_cols] * len(labels_df)


In [29]:
labels_df

,idx_a,idx_b,relative_concept_labels,concept_names
0,88841,197814,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[helpsteer-helpfulness, helpsteer-correctness,..."
1,178251,246231,"[-1.0, -1.0, -1.0, -1.0, -1.0, 0.6000000000000...","[helpsteer-helpfulness, helpsteer-correctness,..."
2,178251,373904,"[-1.0, -1.0, -1.0, -1.0, -1.0, 0.5, 0.7, 0.4, ...","[helpsteer-helpfulness, helpsteer-correctness,..."
3,178251,386973,"[-1.0, -1.0, -1.0, -1.0, -1.0, 0.3500000000000...","[helpsteer-helpfulness, helpsteer-correctness,..."
4,246231,373904,"[-1.0, -1.0, -1.0, -1.0, -1.0, 0.3999999999999...","[helpsteer-helpfulness, helpsteer-correctness,..."
...,...,...,...,...
1019327,73321,186383,"[-1.0, -1.0, -1.0, -1.0, -1.0, 0.4499999999999...","[helpsteer-helpfulness, helpsteer-correctness,..."
1019328,73321,230013,"[-1.0, -1.0, -1.0, -1.0, -1.0, 0.5, 0.5, 0.300...","[helpsteer-helpfulness, helpsteer-correctness,..."
1019329,77062,186383,"[-1.0, -1.0, -1.0, -1.0, -1.0, 0.5, 0.5, 0.5, ...","[helpsteer-helpfulness, helpsteer-correctness,..."
1019330,77062,230013,"[-1.0, -1.0, -1.0, -1.0, -1.0, 0.55, 0.5, 0.5,...","[helpsteer-helpfulness, helpsteer-correctness,..."


In [ ]:
labels_ds = Dataset.from_pandas(labels_df)
labels_ds.save_to_disk(f"{root}/datasets/armo-rm/concept_labels/armo-rm")

In [31]:
labels_df = data_labels.to_pandas()[['idx_a', 'idx_b', 'preference_label']].copy()

In [32]:
labels_df

,idx_a,idx_b,preference_label
0,88841,197814,0
1,178251,246231,0
2,178251,373904,0
3,178251,386973,0
4,246231,373904,1
...,...,...,...
1019327,73321,186383,0
1019328,73321,230013,0
1019329,77062,186383,0
1019330,77062,230013,1


In [ ]:
# Save to disk
labels_ds = Dataset.from_pandas(labels_df)
labels_ds.save_to_disk(f"{root}/datasets/armo-rm/preference_labels/armo-rm")

In [34]:
pairs_df = data_labels.to_pandas()[['idx_a', 'idx_b']].copy()

In [35]:
print(len(pairs_df))

1019332


In [36]:
import numpy as np
splits_df = pairs_df[['idx_a', 'idx_b']].copy()
splits_df['split'] = np.random.choice(['train', 'val', 'test'], p=[0.7, 0.1, 0.2])
splits_df.to_csv(f"{root}/datasets/armo-rm/splits.csv", index=False)

In [37]:
splits_df = pairs_df[['idx_a', 'idx_b']].copy()
splits_df = splits_df.sample(frac=0.1, random_state=42)
splits_df['split'] = np.random.choice(['train', 'val', 'test'], p=[0.7, 0.1, 0.2])
splits_df.to_csv(f"{root}/datasets/armo-rm/small_splits.csv", index=False)